In [ ]:
from pyiceberg.catalog.sql import SqlCatalog
import pyarrow.parquet as pq
import pyarrow.compute as pc
#import sqlite3

#con = sqlite3.connect("pyiceberg_catalog.db")

warehouse_path = "./data"
catalog = SqlCatalog(
    "default",
    **{
        "uri": f"sqlite:///{warehouse_path}/pyiceberg_catalog.db",
        "warehouse": f"file://{warehouse_path}",
    },
)

catalog.create_namespace("default")


In [ ]:
import sqlite3
con = sqlite3.connect("./data/pyiceberg_catalog.db")
cur = con.cursor()
cur.execute("""CREATE TABLE iceberg_namespace_properties (
	catalog_name VARCHAR(255) NOT NULL, 
	namespace VARCHAR(255) NOT NULL, 
	property_key VARCHAR(255) NOT NULL, 
	property_value VARCHAR(1000) NOT NULL, 
	PRIMARY KEY (catalog_name, namespace, property_key)
)""")
con.close()

In [ ]:
df = pq.read_table("./data/taxi-tripdata.parquet")


table = catalog.create_table(
    "default.taxi_dataset",
    schema=df.schema,
)

table.append(df)
len(table.scan().to_arrow())

print(table.scan().to_arrow())